### Roland Farrell 08/02/22
### see arXiv:2207.01731

In [1]:
import numpy as np
import qiskit
from qiskit.quantum_info import Pauli
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.opflow import (I, X, Y, Z, PauliOp, CircuitOp, PauliExpectation, AerPauliExpectation)
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import *
from qiskit.utils import QuantumInstance
from qiskit import Aer, transpile

# Using the Variational Quantum Eigensolver (VQE) to determine the ground state of one flavor QCD on a $L=1$ one dimensional lattice. This system maps onto six qubits.

# Constructing the Hamiltonian

### Returns an operator corresponding to a given Pauli string. The given Pauli string is in the form of a dictionary which contains keys which label the qubit (0-5) and values which are Pauli operators. Factors of the identity are automatically included.

In [2]:
def Hpauli(dict):
    prev = 6
    op = 1
    for index, Pauli in sorted(dict.items(),reverse=True):
        if (prev - index - 1) != 0:
            op = op^(I.tensorpower(prev - index - 1))
        op = op^Pauli
        prev = index
    if prev != 0:
        op = op.tensor(I.tensorpower(prev))
    return op

### Constructs the mass part of the Hamiltonian, returns a PauliSumOp which is a sum of Pauli strings.

In [3]:
def Hm():
    op = 3 * (I^(6))
    for p in range(2):
        for q in range(3):
            op = op + (-1)**(p)/2 * Hpauli({3*p + q: Z})
    return op.reduce()  

### Constructs the kinetic part of the Hamiltonian, returns a PauliSumOp which is a sum of Pauli strings.

In [4]:
def Hkin():
    op = 0
    for p in range(1):
        op = op + 1/2 * (Hpauli({3*p: X, 3*p+1: Z, 3*p+2: Z, 3*p+3: X}) + Hpauli({3*p: Y, 3*p+1: Z, 3*p+2: Z, 3*p+3: Y}))
        op = op + 1/2 * (Hpauli({3*p+1: X, 3*p+2: Z, 3*p+3: Z, 3*p+4: X}) + Hpauli({3*p+1: Y, 3*p+2: Z, 3*p+3: Z, 3*p+4: Y}))
        op = op + 1/2 * (Hpauli({3*p+2: X, 3*p+3: Z, 3*p+4: Z, 3*p+5: X}) + Hpauli({3*p+2: Y, 3*p+3: Z, 3*p+4: Z, 3*p+5: Y}))
    return (1/2 * op).reduce()  

### Constructs the electric part of the Hamiltonian, returns a PauliSumOp which is a sum of Pauli strings.

In [5]:
def Hel():
    op = 3 * (I^(6))
    for p in range(2):
        op = op - (1-p) * (Hpauli({3*p: Z, 3*p+1: Z}) + Hpauli({3*p: Z, 3*p+2: Z}) + Hpauli({3*p+1: Z, 3*p+2: Z}))
    return ((1/ 6)* op).reduce()

### The full Hamiltonian. Takes as parameters the mass, m, and gauge coupling, g. Returns a PauliSumOp which is a sum of Pauli strings.

In [6]:
def H(m,g):
    return (m * Hm() + Hkin() + g**2 * Hel()).reduce()

# Constructing the variational circuit and Hamiltonian
### The energy of the most general state with the quantum numbers of the vacuum can be parameterized by a set of angles $\theta$ as:
###                 $E(\theta) \ = \ \langle{{\Omega_0}}\rvert U_{var}^{\dagger}(\theta) \tilde{H} U_{var}(\theta) \lvert{{\Omega_0}}\rangle$
### where $\lvert{{\Omega_0}}\rangle$ is the trivial vacuum, $U_{var}(\theta)$ is a variational circuit and $\tilde{H}= U_s^{\dagger} H U_s$. Factoring out the static part of the variational circuit, $U_s$, reduces the complexity of the circuits which are run on the quantum device, at the cost of more correlated measurements. By minimizing $E(\theta)$, the circuit which prepares the ground state, $U_s\,U_{var}(\theta)$, is determined. 

### Constructs $U_s$, returns in CircuitOp form

In [7]:
def Us():
    circ = QuantumCircuit(6)
    circ.x(0)
    circ.x(1)
    circ.x(2)
    circ.cx(4,5)
    circ.cx(3, 0)
    circ.cx(3,4)
    circ.cx(4, 1)
    circ.cx(5, 2)
    return CircuitOp(circ)

### Constructs $\tilde{H}$, returns as a PauliSumOp

In [8]:
def Htilde(m,g):
    stat = Us()
    op = 0*(I^6)
    # conjugte each pauli string in H by Us. Also converting to a PauliOp
    for pauli, coeff in H(m,g).primitive.label_iter():
        op = op + (stat.adjoint() @ PauliOp(Pauli(pauli),coeff) @ stat).to_pauli_op()
    op = op.reduce()
    return op

### The variational part of VQE circuit, see Fig. 17 in arXiv:2207.01731 

In [9]:
def VacAnsatz3angleNoStat():
    params = ParameterVector('theta', length=3)
    it = iter(params)
    ansatz = QuantumCircuit(3 * 2)
    
    # Generic B=0 state is parameterized by 7 angles. Reducing to color singlets reduces this to 3 angles
    theta=next(it)
    theta01=next(it)
    theta011=next(it)
    
    theta00 = -2 * np.arcsin(np.tan(theta/2) * np.cos(theta01/2))
    theta001 = -2 * np.arcsin(np.cos(theta011/2) * np.tan(theta01/2))
    theta000 = -2 * np.arcsin(np.tan(theta00/2) * np.cos(theta001/2))
    
    ansatz.ry(theta, 3)
    
    ansatz.x(3)
    ansatz.cry(theta00,3, 4)
    ansatz.x(3)
    
    ansatz.ry(theta01/2,4)
    ansatz.cx(3,4)
    ansatz.ry(-theta01/2,4)
    
    ansatz.ry(theta011/4,5)
    ansatz.cx(3,5)
    ansatz.ry(-theta011/4,5)
    ansatz.cx(4,5)
    ansatz.ry(-theta011/4,5)
    ansatz.cx(3,5)
    ansatz.ry(theta011/4 -theta001/2,5)

    ansatz.cx(4,5)
    ansatz.ry(theta001/2 + theta000/4,5)

    ansatz.cx(3,5)
    ansatz.ry(theta000/4,5)
    ansatz.cx(4,5)
    ansatz.ry(theta000/4,5)
    ansatz.cx(3,5)
    ansatz.ry(theta000/4,5)
    
    # Prepares the most general state with Us factored out.
    
    return ansatz

# Find ground state with qiskit's implementation of VQE

In [10]:
# The SLSQP optimizer works well when running on a quantum device. The CG optimizer is better if using the noiseless Aer simulator.
optimizer = CG(maxiter=50)

# Change backend to put on a real device
qi_sv = QuantumInstance(Aer.get_backend('aer_simulator'),shots=1000)

# PauliExpecation groups measurements into mutually commuting sets. Inital_point takes the initial state as the trivial vacuum.
vqe = VQE(VacAnsatz3angleNoStat(), optimizer = optimizer, expectation = AerPauliExpectation(), initial_point=[0,0,0], quantum_instance = qi_sv)
result = vqe.compute_minimum_eigenvalue(Htilde(1,1))
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 196,
    'eigenstate': {   '000000': 0.9612491872558333,
                      '100000': 0.15491933384829668,
                      '110000': 0.15811388300841897,
                      '111000': 0.16431676725154984},
    'eigenvalue': (-0.2743611471958559+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   ParameterVectorElement(theta[0]): -0.365135601994009,
                              ParameterVectorElement(theta[1]): -0.41168864451967546,
                              ParameterVectorElement(theta[2]): -0.4691389342916442},
    'optimal_point': array([-0.3651356 , -0.41168864, -0.46913893]),
    'optimal_value': -0.2743611471958559,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': 0.24194884300231934}
